# BigQuery SQL

```
SELECT * 
FROM `ftds-hacktiv8-project.phase1_ftds_016_rmt.credit-card-information`
WHERE MOD(CUST_ID,2) = 0
```

# Problems

Buatlah model clustering untuk melakukan Customer Segmentation dari data kartu kredit sebuah bank dibawah ini. Data ini merupakan data informasi penggunaan kartu kredit selama 6 bulan terakhir. 

# Conceptual Problems

1. Apakah yang dimaksud dengan `inertia` pada algoritma K-Means ?

Inertia, a `sklearn.cluster.KMeans` attribute, is sum of squared distances of samples to their closest cluster center. The formula looks like this `np.sum( ( Euclidean distance point 1 )**2 + ... + ( Euclidean distance point N )**2 )`, euclidean distance is `np.sqrt( x**2 + y**2 )`.

In other words, the farther a point is to the center, the higher the inertia is. If we visualize it with a scatter plot, we will see the points to be scattered instead of clusterin near the centroid. Therefore, we can say, the higher the interia, the more variations there are.

Therefore, we can use Inertia to determine the best clustering (`sklearn.cluster.KMeans` n_clusters parameter). One way to do this is by visualizing it into line plot (x = number of custers, y = inertia). Simply find the elbow within the plot to find the optimal number of clusters.

2. Jelaskan yang dimaksud dengan Elbow Method (alasan penggunaan, cara penggunaan, kelemahan/kelebihan, dll) !

Elbow method relies on the inertia score. Inertia score explains the total variations within a cluster. In other words, the farther a point to its centroid, the higher the variations is. If do a line plot with the inertia score, we will see an elbow. The elbow tells us that the reduction in variations after the elbow is insignificant.

The reason we use Elbow method: We use Elbow method because we want to reduce the variations within each cluster.

How to use Elbow method: We can use Elbow method by plotting the `sklearn.cluster.KMeans` inertia_ attribute as the y-axis and the number of clusters as the x-axis.

The weakness of Elbow method: It does not take into account whether a point has been assigned to the wrong cluster. In other words, inertia score incudes `np.sum ( Eucledian distance point outlier )**2 `. Therefore, we may deem that the inertia score after the elbow to be insignificant when in fact, there are still points that have been assigned to the wrong cluster (because the decision boundary overlaps with another cluster)

The strength: It is easy to interpret if we compared it to the silhouette score visualization. Just one graph and we are good to go. If we use silhouette score, we need to visualize the silhouette score against number of clusters. Then, we need to visualize the silhouette score against the coefficient. Then, we need to compare it between each plot (which is the plot that do not have minus silhouette coefficient (we calculate the silhouette coefficient for each point! Therefore we can know whether a point hve minus silhouette coefficient or not), which is the plot that have cluster roughly the same density, which is the plot that all cluster silhouette coefficient pass the silhouette score).

3. Jelaskan perbedaan masing-masing algoritma clustering yang anda gunakan dalam kasus ini !

We are going to use KMeans, DBScan and Spectral Clustering.

KMeans uses the Eucledian Distance to determine whether a point is part of a cluster or not by calculating the distance of a point to every centroid. Then, the point will be assigned to the cluster with the closest distance.

DBScan uses the distance between a point and centroid (we call it as epsilon) and a miinmum samples to determine whether a point is part of a cluster or an outlier.

Spectral Clustering uses the kernel function (e.g RBF) to determine the relationship between a point to a point. Then, the algorithm assign that point to a cluster that have strongest relationship with that point.

# 1 - Perkenalan

Name : Jason Rich Darmawan Onggo Putra

Dataset Overview: 

Objective:

**Features**

<table style="width: 85%;">
<tr>
  <th>Feature</th>
  <th>Description</th>
</tr>

<tr>
  <td>CUST_ID</td>
  <td>Identification of Credit Card holder</td>
</tr>

<tr>
  <td>BALANCE</td>
  <td>Balance amount left in their account to make purchases</td>
</tr>

<tr>
  <td>BALANCE_FREQUENCY</td>
  <td>How frequently the Balance is updated, score between 0 and 1 (1 = frequently updated, 0 = not frequently updated)</td>
</tr>

<tr>
  <td>PURCHASES</td>
  <td>Amount of purchases made from account</td>
</tr>

<tr>
  <td>ONEOFF_PURCHASES</td>
  <td>Maximum purchase amount done in one-go</td>
</tr>

<tr>
  <td>INSTALLMENTS_PURCHASES</td>
  <td>Amount of purchase done in installment</td>
</tr>

<tr>
  <td>CASH_ADVANCE</td>
  <td>Cash in advance given by the user</td>
</tr>

<tr>
  <td>PURCHASES_FREQUENCY</td>
  <td>How frequently the Purchases are being made, score between 0 and 1 (1 = frequently purchased, 0 = not frequently purchased)</td>
</tr>

<tr>
  <td>ONEOFF_PURCHASES_FREQUENCY</td>
  <td>How frequently Purchases are happening in one-go (1 = frequently purchased, 0 = not frequently purchased)</td>
</tr>

<tr>
  <td>PURCHASES_INSTALLMENTS_FREQUENCY</td>
  <td>How frequently purchases in installments are being made (1 = frequently done, 0 = not frequently done)</td>
</tr>

<tr>
  <td>CASH_ADVANCE_FREQUENCY</td>
  <td>How frequently the cash in advance being paid</td>
</tr>

<tr>
  <td>CASH_ADVANCE_TRX</td>
  <td>Number of Transactions made with 'Cash in Advanced'</td>
</tr>

<tr>
  <td>PURCHASES_TRX</td>
  <td>Number of purchase transactions made</td>
</tr>

<tr>
  <td>CREDIT_LIMIT</td>
  <td>Limit of Credit Card for user</td>
</tr>

<tr>
  <td>PAYMENTS</td>
  <td>Amount of Payment done by user</td>
</tr>

<tr>
  <td>MINIMUM_PAYMENTS</td>
  <td>Minimum amount of payments made by user</td>
</tr>

<tr>
  <td>PRC_FULL_PAYMENT</td>
  <td>Percent of full payment paid by user</td>
</tr>

<tr>
  <td>TENURE</td>
  <td>Tenure of credit card service for user</td>
</tr>
</table>

# 2 - Import Libraries

In [4]:
# Data Loading
import pandas as pd

# Exploratory Data Analysis
import numpy as np

# 3 - Data Loading

TODO
- [ ] Drop column 'CUST_ID' because it should not have collinearity with other features.
- Every features in the data set are numerical.

In [5]:
# Load dataset
dt = pd.read_csv("./h8dsft_P1G4_jason_rich_darmawan_onggo_putra.csv")

# Data set preview
with pd.option_context("display.float_format", "{:,.2f}".format):
    display(dt)

,CUST_ID,BALANCE,BALANCE_FREQUENCY,PURCHASES,ONEOFF_PURCHASES,INSTALLMENTS_PURCHASES,CASH_ADVANCE,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,PURCHASES_INSTALLMENTS_FREQUENCY,CASH_ADVANCE_FREQUENCY,CASH_ADVANCE_TRX,PURCHASES_TRX,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT,TENURE
0,788,16.58,0.12,200.00,0.00,200.00,0.00,0.12,0.00,0.12,0.00,0,2,"1,200.00",0.00,NaN,0.00,8
1,1902,943.58,0.88,103.00,103.00,0.00,0.00,0.12,0.12,0.00,0.00,0,1,"1,000.00",215.33,410.48,0.00,8
2,2322,1.98,0.12,80.00,80.00,0.00,0.00,0.12,0.12,0.00,0.00,0,1,"1,500.00",0.00,NaN,0.00,8
3,3426,461.46,1.00,585.00,585.00,0.00,0.00,0.12,0.12,0.00,0.00,0,1,"1,000.00",258.03,151.70,0.00,8
4,3976,"1,053.59",1.00,"1,450.00","1,450.00",0.00,0.00,0.12,0.12,0.00,0.00,0,1,"1,500.00",397.09,227.14,0.00,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4470,4630,"1,079.01",1.00,0.00,0.00,0.00,"1,259.57",0.00,0.00,0.00,0.55,9,0,"1,200.00","1,379.64",271.36,0.09,11
4471,8614,809.58,0.91,0.00,0.00,0.00,"3,819.67",0.00,0.00,0.00,0.55,7,0,"4,150.00","4,372.15",241.65,0.09,11
4472,4296,"3,386.49",1.00,560.98,427.22,133.76,"2,008.15",1.00,0.73,0.36,0.55,9,20,"10,100.00","1,039.00",886.05,0.00,11
4473,4476,"4,840.55",1.00,"1,595.93","1,595.93",0.00,"4,359.33",0.45,0.45,0.00,0.55,17,25,"6,000.00",905.84,"2,376.92",0.09,11


TODO
- [ ] Handle missing values on feature 'MINIMUM_PAYMENTS'.
- [ ] Drop missing values from feature 'CREDIT_LIMIT'.

Duly noted: We will handle these in [Data Preprocessing](#5---data-preprocessing)

In [31]:
# Check features dtype
dt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4475 entries, 0 to 4474
Data columns (total 17 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   BALANCE                           4475 non-null   float64
 1   BALANCE_FREQUENCY                 4475 non-null   float64
 2   PURCHASES                         4475 non-null   float64
 3   ONEOFF_PURCHASES                  4475 non-null   float64
 4   INSTALLMENTS_PURCHASES            4475 non-null   float64
 5   CASH_ADVANCE                      4475 non-null   float64
 6   PURCHASES_FREQUENCY               4475 non-null   float64
 7   ONEOFF_PURCHASES_FREQUENCY        4475 non-null   float64
 8   PURCHASES_INSTALLMENTS_FREQUENCY  4475 non-null   float64
 9   CASH_ADVANCE_FREQUENCY            4475 non-null   float64
 10  CASH_ADVANCE_TRX                  4475 non-null   int64  
 11  PURCHASES_TRX                     4475 non-null   int64  
 12  CREDIT

# 4 - Exploratory Data Analysis

- 'CASH_ADVANCE_FREQUENCY' max value is 1.17, probably an error.
- 'MINIMUM_PAYMENTS' min value is 0.04, probably an error.
- Every numerical columns are either moderately or highly skewed to the right except:
  - 'PURCHASES_FREQUENCY' distribution is approximately symmetric.
  - 'TENURE' distribution is highly skewed to the right.

Duly noted: Regarding handling errors, without a confirmation from the data set owner, we will not handle it.

In [32]:
# Check central tendency
with pd.option_context("display.float_format", "{:.2f}".format):
    display(dt.describe())

,BALANCE,BALANCE_FREQUENCY,PURCHASES,ONEOFF_PURCHASES,INSTALLMENTS_PURCHASES,CASH_ADVANCE,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,PURCHASES_INSTALLMENTS_FREQUENCY,CASH_ADVANCE_FREQUENCY,CASH_ADVANCE_TRX,PURCHASES_TRX,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT,TENURE
count,4475.00,4475.00,4475.00,4475.00,4475.00,4475.00,4475.00,4475.00,4475.00,4475.00,4475.00,4475.00,4474.00,4475.00,4317.00,4475.00,4475.00
mean,1565.74,0.88,1003.16,588.50,415.03,973.45,0.49,0.20,0.36,0.13,3.20,14.49,4494.02,1726.21,854.43,0.16,11.52
std,2091.21,0.24,2210.81,1666.50,927.36,2133.46,0.40,0.30,0.40,0.20,6.50,24.16,3668.54,2961.26,2306.41,0.30,1.35
min,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,150.00,0.00,0.04,0.00,6.00
25%,132.54,0.89,40.06,0.00,0.00,0.00,0.08,0.00,0.00,0.00,0.00,1.00,1500.00,390.21,169.81,0.00,12.00
50%,874.91,1.00,362.48,29.99,89.45,0.00,0.50,0.08,0.17,0.00,0.00,7.00,3000.00,831.05,309.85,0.00,12.00
75%,2058.52,1.00,1122.64,580.32,484.25,1098.57,0.92,0.33,0.75,0.18,4.00,17.00,6000.00,1895.96,830.40,0.17,12.00
max,19043.14,1.00,49039.57,40761.25,22500.00,29282.11,1.00,1.00,1.00,1.17,110.00,358.00,30000.00,46930.60,61031.62,1.00,12.00


TODO
- [ ] Handle natural outliers because Principal Component Analysis is very sensitive to outliers and can lead to misleading conclusions in the presence of outliers.

We will handle these in [Data Preprocessing](#5---data-preprocessing)

In [33]:
# Check skewness.
dt.skew()

BALANCE                              2.456621
BALANCE_FREQUENCY                   -2.020309
PURCHASES                            8.987052
ONEOFF_PURCHASES                    10.140059
INSTALLMENTS_PURCHASES               8.116887
CASH_ADVANCE                         4.739678
PURCHASES_FREQUENCY                  0.061598
ONEOFF_PURCHASES_FREQUENCY           1.523455
PURCHASES_INSTALLMENTS_FREQUENCY     0.510288
CASH_ADVANCE_FREQUENCY               1.823380
CASH_ADVANCE_TRX                     4.752780
PURCHASES_TRX                        4.480452
CREDIT_LIMIT                         1.545078
PAYMENTS                             6.040773
MINIMUM_PAYMENTS                    12.357860
PRC_FULL_PAYMENT                     1.911650
TENURE                              -2.932822
dtype: float64

# 5 - Data Preprocessing

In [6]:
# Drop column 'CUST_ID'
dt.drop(labels='CUST_ID', axis=1, inplace=True, errors='ignore')

In [15]:
# Drop missing values from feature 'CREDIT_LIMIT'
dt.dropna(axis=0, subset='CREDIT_LIMIT', inplace=True)

In [10]:
# Handle missing values on feature 'MINIMUM_PAYMENTS'
dt.loc[dt['MINIMUM_PAYMENTS'].isna(),['PAYMENTS', 'MINIMUM_PAYMENTS']].index

Int64Index([   0,    2,    6,    8,   27,   32,   78,   92,   93,   94,
            ...
            4014, 4017, 4021, 4029, 4040, 4049, 4087, 4091, 4144, 4293],
           dtype='int64', length=158)